In [6]:
import os
import json
import time
import requests
from datetime import datetime
import pandas as pd
import numpy as np

from azure.ai.ml import MLClient
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    Model,
    Environment,
    CodeConfiguration
)
from azure.ai.ml.constants import AssetTypes
from azure.identity import DefaultAzureCredential

import matplotlib.pyplot as plt

In [7]:
SUBSCRIPTION_ID = os.getenv("AZURE_SUBSCRIPTION_ID", "3fc7fd13-533e-40a7-8e3d-f1fbf4204436")
RESOURCE_GROUP = os.getenv("AZURE_RESOURCE_GROUP", "edu-demo")
WORKSPACE_NAME = os.getenv("AZURE_WORKSPACE_NAME", "edu-demo")

try:
    credential = DefaultAzureCredential()
    ml_client = MLClient(
        credential=credential,
        subscription_id=SUBSCRIPTION_ID,
        resource_group_name=RESOURCE_GROUP,
        workspace_name=WORKSPACE_NAME,
    )
    
    print("Azure ML Client initialized successfully!")
    print(f"Workspace: {ml_client.workspace_name}")
    
except Exception as e:
    print(f"Error initializing Azure ML Client: {e}")

Azure ML Client initialized successfully!
Workspace: edu-demo


In [8]:
try:
    # Get the latest version of our registered model
    model_name = "auto-registered-classification-model"
    model = ml_client.models.get(model_name, version="1")
    
    print(f"Retrieved model: {model.name}:{model.version}")
    print(f"Model description: {model.description}")
    print(f"Model tags: {model.tags}")
    
    model_available = True
    
except Exception as e:
    print(f"Error retrieving model: {e}")
    print("Please ensure you have registered a model in notebook 2.")
    model_available = False

Retrieved model: auto-registered-classification-model:1
Model description: Auto-registered model from job silver_knee_7bhcyl7021
Model tags: {'training_job': 'silver_knee_7bhcyl7021', 'framework': 'scikit-learn', 'algorithm': 'random_forest', 'auto_registered': 'true', 'training_date': '2025-07-19 04:27:25'}


In [9]:
try:
    # Create inference environment
    inference_env = Environment(
        name="inference-env",
        description="Environment for model inference",
        conda_file="./deployment/src/conda.yml",
        image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:latest"
    )
    
    # Create or update the environment
    inference_env = ml_client.environments.create_or_update(inference_env)
    print(f"Inference environment '{inference_env.name}' created successfully!")
    
except Exception as e:
    print(f"Error creating environment: {e}")
    print("Will attempt to use existing environment.")

Inference environment 'inference-env' created successfully!


In [10]:
if model_available:
    try:
        # Define endpoint name (must be unique)
        import uuid
        endpoint_name = f"synthetic-{str(uuid.uuid4())[:8]}"
        
        # Create endpoint
        endpoint = ManagedOnlineEndpoint(
            name=endpoint_name,
            description="Endpoint for synthetic classification model",
            auth_mode="key",  # Use key-based authentication
            tags={
                "model_name": model_name,
                "purpose": "demo",
                "created_by": "azure-ml-notebook"
            }
        )
        
        # Create the endpoint
        print(f"Creating endpoint: {endpoint_name}")
        endpoint = ml_client.online_endpoints.begin_create_or_update(endpoint).result()
        
        print(f"Endpoint created successfully!")
        print(f"Endpoint name: {endpoint.name}")
        print(f"Endpoint URI: {endpoint.scoring_uri}")
        
    except Exception as e:
        print(f"Error creating endpoint: {e}")
        print("\nPossible issues:")
        print("1. Endpoint name already exists (try with different name)")
        print("2. Insufficient permissions")
        print("3. Quota limitations")
else:
    print("Cannot create endpoint without a registered model.")

Creating endpoint: synthetic-fdd4a3b4


KeyboardInterrupt: 

In [12]:
# Get endpoint details
endpoint_details = ml_client.online_endpoints.get("synthetic-9f4e14c2")
scoring_uri = endpoint_details.scoring_uri

# Get authentication key
keys = ml_client.online_endpoints.get_keys(endpoint_name)
auth_key = keys.primary_key

print(f"Endpoint URI: {scoring_uri}")
print(f"Auth key: {auth_key[:10]}...")

# Test with sample data
test_data = {
    "data": [
        [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0,
            1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2.0],
        [-0.1, -0.2, -0.3, -0.4, -0.5, -0.6, -0.7, -0.8, -0.9, -1.0,
            -1.1, -1.2, -1.3, -1.4, -1.5, -1.6, -1.7, -1.8, -1.9, -2.0]
    ]
}

# Make request
headers = {
    'Content-Type': 'application/json',
    'Authorization': f'Bearer {auth_key}'
}

print("\nTesting endpoint...")
response = requests.post(
    scoring_uri,
    json=test_data,
    headers=headers,
    timeout=30
)

if response.status_code == 200:
    result = response.json()
    print("✅ Endpoint test successful!")
    print("Response:")
    print(json.dumps(result, indent=2))
else:
    print(f"❌ Endpoint test failed: {response.status_code}")
    print(f"Error: {response.text}")
    

Endpoint URI: https://synthetic-9f4e14c2.westeurope.inference.ml.azure.com/score
Auth key: 7ZlgB7KnhH...

Testing endpoint...
❌ Endpoint test failed: 401
Error: key_auth_access_denied
Please check this guide to understand why this error code might have been returned 
https://docs.microsoft.com/en-us/azure/machine-learning/how-to-troubleshoot-online-endpoints#http-status-codes

